# Create MPS 

In [15]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
% matplotlib inline
from IPython.display import Latex
import math
#import scipy.special
#import scipy.constants as sc
from scipy import sparse as sparse
from scipy.sparse import linalg
import gc

### Create a random MPS for sites 1-L, of local dimension d and of mximal bond dimension D

In [2]:
# Define the dagger function (transpose conjugate is too long to type everytime)
def dagger(A):
    return np.conjugate(np.transpose(A))

In [17]:
def create_random_mps(L,D,d):
    mps = [0]*(L+1)                    # create empty list of fixed length!
    mps[1] = np.random.rand(1,d,D)     # on site 1 and L we have d (1xD) and (Dx1) matrices to get a scalar
    mps[L] = np.random.rand(D,d,1)     # quantum amplitude
    for i in range(2, L):
        mps[i] = np.random.rand(D,d,D) # at each site i create tensor with dimensions D,d,D i.e. d (DxD) matrices
    return mps

### Normalise the MPS on a specific site k, with option of right ('R') or left ('L') normalisation and input mps 

##### NOTE: For right canonical state need to normalise starting at site L, for left canonical at site 1 !!!!

In [18]:
def mpsnormalisation(mps, LR, k):
    d = mps[k].shape     # get the dimensions of the mps/tensor train
    if LR == 'L':               # left-normalise mps at site k i.e. M(k)M(k+1) = USVM(k+1) = A(k)SVM(k+1) = A(k)M(k+1)
        if k < len(mps) - 1:
            M = mps[k].reshape(d[0]*d[1],d[2])          #reshape 3 tensor into matrix
            U, s, V = LA.svd(M, full_matrices=False)    # perform SVD decomposition on reshaped tensor
            S = np.diag(s)
            mps[k] = U.reshape(d[0],  d[1], U.shape[1]) # reassign reshaped U as newly normalised A_L matrix 
            mps[k+1] = np.tensordot(np.dot(S,V),mps[k+1], axes = ([1],[0])) #SV are tensormultiplied into M(k+1)
        elif k == len(mps)-1:
            M = mps[k].reshape(d[0]*d[1],d[2])          #reshape 3 tensor into matrix
            U, s, V = LA.svd(M, full_matrices=False)    # perform SVD decomposition on reshaped tensor
            S = np.diag(s)
            ## C = np.dot(S,V)
            ## C is 1x1 matrix i.e. a scalar that represents the norm of the state, so we can ignore C at the last 
            ## site L and ONLY assign U from SVD further to A_L to get normalised state.
            mps[k] = U.reshape(d[0],  d[1], U.shape[1])# reassign reshaped U as newly normalised A_L matrix 
    elif LR == 'R':            # right-normalise mps at site l i.e. M(l-1)M(l) = M(l-1)USV = M(l-1)USB(l) = M(l-1)B(l)
        if k > 1:
            M = mps[k].reshape(d[0],d[1]*d[2])
            U, s, V = LA.svd(M, full_matrices=False)
            S = np.diag(s)
            mps[k] = V.reshape(V.shape[0],d[1],d[2])
            mps[k-1] = np.tensordot(mps[k-1],np.dot(U,S), axes = ([2],[0])) # last index of M(k-1) is multiplied into 
                                                                            # first of (US)
        elif k == 1:
            M = mps[k].reshape(d[0],d[1]*d[2])          #reshape 3 tensor into matrix
            U, s, V = LA.svd(M, full_matrices=False)    # perform SVD decomposition on reshaped tensor
            S = np.diag(s)
            ## C = np.dot(U,S)
            ## C is 1x1 matrix i.e. a scalar that represents the norm of the state, so we can ignore C at the first 
            ## site 1 and ONLY assign V from SVD further to B_1 to get normalised state.
            mps[k] = V.reshape(V.shape[0],  d[1], d[2]) # reassign reshaped U as newly normalised A_L matrix 
    else:
        print('Input L or R for left-/right normalisation')
        #return mps

In [19]:
def mixedmps(mps,k):
    for i in range(1,k):
        mpsnormalisation(mps,'L',i)            # left normalise up to site k-1
    for i in range(1,len(mps) - (k+2) + 1):    # Isolate the sites k and k+1 which are in the different A/B blocks
        mpsnormalisation(mps,'R',len(mps)-i)   # right normalise down to site k+2, starting from site L
    C = np.tensordot(mps[k],mps[k+1], axes([2],[0]))
    d = C.shape
    M = C.reshape(d[0]*d[1],d[2]*d[3])
    U, s, V = LA.svd(M, full_matrices=False)
    mps[k] = U.reshape(d[0],d[1],U.shape(1))
    mps[k+1] = V.reshape(V.shape(0),d[1], d[2])

### Create MPO for the Heisenberg chain

In [20]:
# Create Heisenberg MPO
def Heisenbergmpo(L,d, J, Jz, h):
    # Define the Pauli matrices...only 2x2 because we have local dimension d = 2 in the Heisenberg chain (spin-1/2) 
    # NOTE: spin operators are 1/2*hbar*Paulimatrices!!!!!
    Sx = 0.5*np.matrix([[0,1],[1,0]])
    Sy = 0.5*np.matrix([[0,-1.0j],[1.0j,0]])
    Sz = 0.5*np.matrix([[1,0],[0,-1]])
    Splus  = Sx + 1.0j*Sy
    Sminus = Sx - 1.0j*Sy
    I = np.eye(d)            

    mpo = [0]*(L+1)                          #create empty list for all 1-L physical sites (0 is dummy)
    
    # 5 is the dimension for this specific construction of the Heisenberg model so W is (5x5xdxd) tensor
    # initialise W as zero-tensor...and put in [b_(i-1), b_(i)] components by hand
    W = np.zeros((5,5,d,d), dtype = complex) # define as complex, else imaginary part of Splus/Sminus will be discarded
    
    W[0,0,:,:] = I
    W[1,0,:,:] = Splus
    W[2,0,:,:] = Sminus
    W[3,0,:,:] = Sz
    W[4,0,:,:] = -h*Sz
    
    W[4,1,:,:] = J/2.0*Sminus
    W[4,2,:,:] = J/2.0*Splus
    W[4,3,:,:] = Jz*Sz
    W[4,4,:,:] = I
    
    for i in range(2,L):
        mpo[i] = W

    
    mpo[1] = np.zeros((1,5,d,d), dtype = complex)
    mpo[L] = np.zeros((5,1,d,d), dtype = complex)
    mpo[1][0,:,:,:] = W[4,:,:,:]
    mpo[L][:,0,:,:] = W[:,0,:,:]
    return mpo

In [21]:
# Mpo for local magnetisation
def magnetisation(L, d, n):
    Sz = 0.5*np.matrix([[1,0],[0,-1]])
    mpo = [0]*(L+1)
    W = np.zeros((1,1,d,d,))
    W[0,0,:,:] = np.identity(2)
    S = np.zeros((1,1,d,d,))
    S[0,0,:,:] = Sz
    for i in range(1, L+1):
        if i == n:
            mpo[i] = S
        else:
            mpo[i] = W
    return mpo

In [22]:
M = magnetisation(3,2,2)
m = M[1]
print(m.shape)
m = np.tensordot(m,M[2], axes = ([1],[0]))
print(m.shape)
m = np.tensordot(m,M[3], axes = ([3],[0]))
print(m.shape)
m = np.swapaxes(m, 2,3)
m = np.swapaxes(m,3,6)
m = np.swapaxes(m,6,4)
print(m.shape)
d = m.shape
m = m.reshape(d[0]*d[1]*d[2]*d[3], d[4]*d[5]*d[6]*d[7])
print(dagger(m) == m)
#d = m.shape
#m = m.reshape(d[0]*d[1]*d[3], d[2]*d[5]*d[4], d[6], d[7])

(1, 1, 2, 2)
(1, 2, 2, 1, 2, 2)
(1, 2, 2, 2, 2, 1, 2, 2)
(1, 2, 2, 2, 2, 1, 2, 2)
[[ True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True]]


### Create the L and R tensors for the eigenvalue equation 

In [23]:
def initialiseLR(mps, mpo):
    length = len(mps) # get the lenght of mps/mpo i.e. number of sites
    
    # define and initialise the F tensors to iteratively construct and update the LR tensors
    F = [0]*(length+1) # create empty list of F tensors
    F[0] = np.ones((1,1,1)) # initialise the first L tensor for convenience to 1
    F[length] = np.ones((1,1,1)) # initialise the last R tensor for convenience to 1
    for i in range(1,length):
        # initialise the F tensors in the right shape
        F[length-i] = np.zeros((mps[length-i].shape[2], mpo[length-i].shape[1], mps[length-i].shape[2]))
        
        #do the tensorproducts with optimal bracketing one after another: FB, WFB, F = BWFB 
        #FB  = np.zeros(np.tensordot(F[length+1-i], mps[length-i], axes = ([2],[2])).shape)
        FB  = np.tensordot(F[length+1-i], mps[length-i], axes = ([2],[2]))
        #WFB = np.zeros(np.tensordot(mpo[length-i], FB, axes = ([1,3],[1,3])).shape)
        WFB = np.tensordot(mpo[length-i], FB, axes = ([1,3],[1,3]))
        F[length-i] = np.tensordot(np.conjugate(mps[length-i]), WFB, axes = ([1,2],[1,2]))
    
    return F

In [24]:
def updateLR(F, mps, mpo, k, lr):
    if lr == 'r':
        # do the right step FA => WFA => L = F = AWFA
        #FA  = np.zeros(np.tensordot(F[k-1], mps[k], axes = ([2],[0])).shape)
        FA  = np.tensordot(F[k-1], mps[k], axes = ([2],[0]))
        #WFA = np.zeros(np.tensordot(mpo[k], FA, axes = ([0,3],[1,2])).shape)
        WFA = np.tensordot(mpo[k], FA, axes = ([0,3],[1,2]))
        F[k] = np.tensordot(np.conjugate(mps[k]), WFA, axes = ([0,1],[2,1]))
        
        ## At right step define the new L and R tensor as F[k] and F[k+2]
        L = F[k]
        R = F[k+2] # this should NOT go out of bounds as for right sweep we start at site 1 and go up to site L-1
        return L, R
    elif lr == 'l':
        # do the left step FB => WFB => R = F = BWFB
        #FB  = np.zeros(np.tensordot(F[k+1], mps[k], axes = ([2],[2])).shape)
        FB  = np.tensordot(F[k+1], mps[k], axes = ([2],[2]))
        #WFB = np.zeros(np.tensordot(mpo[k], FB, axes = ([1,3],[1,3])).shape)
        WFB = np.tensordot(mpo[k], FB, axes = ([1,3],[1,3]))
        F[k] = np.tensordot(np.conjugate(mps[k]), WFB, axes = ([1,2],[1,2]))
        
        # At left step define the new L and R tensor as F[k-2] and F[k]
        L = F[k-2] # this should NOT go out of bounds as for left sweep we start at site L and go down to site 2
        R = F[k]
        return L, R

In [48]:
def Heffoptimisation(mps, mpo, L, R, n, lr):
    #WR = np.tensordot(mpo[k],F[k+1], axes = ([1],[1])) # contract W with R first
    #Heff = np.tensordot(F[k-1], WR, axes = ([1],[0]))  # contract L with WR to get Heff = LWR
    WR = np.tensordot(mpo[n],R, axes = ([1],[1])) # contract W with R first
    Heff = np.tensordot(L, WR, axes = ([1],[0]))  # contract L with WR to get Heff = LWR
    
    ### Heff has indices a_(i-1) a'_(i-1) sigma_i sigma'_i a_(i) a'_(i)
    ### Need to be careful to reshape correctly (such that the correct indices are put together)
    ### To achieve this group the indices correctly BEFOREHAND with np.swapaxes
    Heff = np.swapaxes(Heff, 1,2)  # swap  a'_(i-1) <=> sigma_i
    Heff = np.swapaxes(Heff, 2,4)  # swap  a'_(i-1) <=> a_(i)
    Heff = np.swapaxes(Heff, 3,4)  # swap  sigma'_i <=> a'_(i-1)
    p = Heff.shape
    
    # Now reshape accordingly 
    Heff = Heff.reshape(p[0]*p[1]*p[2], p[3]*p[4]*p[5])       # reshape Heff into 2x2 matrix
    
    #yesno = (dagger(Heff) - Heff < 1e-14 )
    #print(np.where(yesno == False))
    
    # Heff is a sparse matrix, so use eigs from sparse rather than LA.eig for speedup
    #### Input initial Guess for speedup!!! ###
    d = mps[n].shape
    initialguess  = mps[n].reshape(d[0]*d[1]*d[2])
    #print initialguess
    E, v = sparse.linalg.eigsh(Heff, k=1, which = 'SA', maxiter=1000000, v0 = initialguess) #return_eigenvectors=True,  v0 = initialguess)
    #E, x = LA.eig(Heff) #, k=1, which = 'SM' v0 = initialguess, return_eigenvectors=True)
    #which = np.argmin(E)
    #v = x[:,which]
    
    if lr == 'r':  # update for a right sweep
        M = v.reshape(p[3]*p[4], p[5])
        U, s, V = LA.svd(M, full_matrices=False)
        S = np.diag(s)
        mps[n] = U.reshape(p[3], p[4], U.shape[1]) # update mps at site k
        mps[n+1] = np.tensordot(np.dot(S,V), mps[n+1], axes = ([1],[0]))
    elif lr == 'l':  # update for a left sweep
        M = v.reshape(p[3], p[4]*p[5])
        U, s, V = LA.svd(M, full_matrices=False)
        S = np.diag(s)
        mps[n] = V.reshape(V.shape[0], p[4], p[5]) # update mps at site k
        mps[n-1] = np.tensordot(mps[n-1], np.dot(U,S), axes = ([2],[0]))
    return E, mps #[which], mps

In [49]:
def localmag(mps, mpo, L, R, n):
    #print(mpo[n].shape, R.shape)
    WR = np.tensordot(mpo[n],R, axes = ([1],[1])) # contract W with R first
    #print(L.shape, WR.shape)
    mag = np.tensordot(L, WR, axes = ([1],[0]))  # contract L with WR to get mag = LWR
    ### Heff has indices a_(i-1) a'_(i-1) sigma_i sigma'_i a_(i) a'_(i)
    ### Need to be careful to reshape correctly (such that the correct indices are put together)
    ### To achieve this group the indices correctly BEFOREHAND with np.swapaxes
    mag = np.swapaxes(mag, 1,2)  # swap  a'_(i-1) <=> sigma_i
    mag = np.swapaxes(mag, 2,4)  # swap  a'_(i-1) <=> a_(i)
    mag = np.swapaxes(mag, 3,4)  # swap  sigma'_i <=> a'_(i-1)
    p = mag.shape
    
    # Now reshape accordingly 
    mag = mag.reshape(p[0]*p[1]*p[2], p[3]*p[4]*p[5])       # reshape mag into 2x2 matrix
    
    yesno = (dagger(mag) - mag < 1e-15 )
    #print(np.where(yesno == False))
    
    d = mps[n].shape
    vector = mps[n].reshape(d[0]*d[1]*d[2],1)
    magnet = np.dot(dagger(vector), np.dot(mag,vector))
    return magnet

In [58]:
# number of sites, sweeps and D cutoff
N = 20   
sweeps = 10 
cutoff = 30
#site = 3

#print('Look at local magnetisation on site ', site)
filename = 'heisenberg_magnetisationD' + str(cutoff) + '.txt'
#print filename
f = open(filename, 'a')

for D in range(30, cutoff + 20, 20):
    cut = '\n' + 'D:' + str(D) + '\n'
    f.write(cut)
    #print D
    
    a = create_random_mps(N, D, 2)

    # Normalise mps
    for i in range(1, len(a)):
        mpsnormalisation(a, 'L', i)
    for i in range(1, len(a)):
        mpsnormalisation(a, 'R', len(a)-i)

    # Create Heisenberg mpo and initialise F (L and R) tensors
    H = Heisenbergmpo(N, 2, 1, 1, 0)
    #M = magnetisation(N, 2, site)
    F = initialiseLR(a, H)
    #f = initialiseLR(a, M)
    
    # Do several sweeps through the sites and optimise mps and energy
    for p in range(1, sweeps+1):
        sweep = 'sweep:' + str(p) + '\n'
        f.write(sweep)
        #print sweep

        for i in range(1, N):
            #print i
            if i == 1:
                L = F[0]
                R = F[2]
                #left = f[0]
                #right = f[2]

            #if i == site:
            #    magnet = localmag(a, M, left, right, i)
            #    if np.absolute(np.imag(magnet[0,0])) < 1e-16:
            #        print('magnetisation:', np.real(magnet[0,0]))
            #    else:
            #        print('magnetisation:', magnet[0,0])
                
            E, a = Heffoptimisation(a, H, L, R, i, 'r')  # right optimisation
            L, R = updateLR(F, a, H, i, 'r')
            #left, right = updateLR(f, a, M, i, 'r')

        rightE = 'right: ' + str(float(E)) + '\n'
        f.write(rightE)
        #print rightE

        for i in range(1, N):
            m = N+1-i
            #print m
            #if m == site:
            #    magnet = localmag(a, M, left, right, m)
            #    print('magnetisation:', magnet)
                
            E, a = Heffoptimisation(a, H, L, R, m, 'l')  # left optimisation
            L, R = updateLR(F, a, H, m, 'l')
            #left, right = updateLR(f, a, M, m, 'l')

        leftE = 'left : ' + str(float(E)) + '\n'
        f.write(leftE)
        #print leftE
        del L, R
        gc.collect()

Sz = 0.5*np.matrix([[1,0],[0,-1]])
for i in range(1, N+1):
    #print i
    if i == 1:
        L = F[0]
        R = F[2]
        #left = f[0]
        #right = f[2]

    #if i == site:
    #    magnet = localmag(a, M, left, right, i)
    #    if np.absolute(np.imag(magnet[0,0])) < 1e-16:
    #        print('magnetisation:', np.real(magnet[0,0]))
    #    else:
    #        print('magnetisation:', magnet[0,0])
    
    MM = np.tensordot(a[i], np.conjugate(a[i]), axes=([0, 2], [0, 2]))
    magnetisation = np.tensordot(Sz, MM, axes=([0,1], [1, 0]))
    if np.imag(magnetisation) < 1e-17:
        f.write('Magnetisation on site ' + str(i) + ' is m = ' + str(float(np.real(magnetisation))) + '\n')
    else:
        f.write('Magnetisation on site ' + str(i) + ' is m = ' + str(magnetisation) + '\n')
    
    if i < N:
        E, a = Heffoptimisation(a, H, L, R, i, 'r')  # right optimisation
        L, R = updateLR(F, a, H, i, 'r')
        #left, right = updateLR(f, a, M, i, 'r')

f.close()

heisenberg_magnetisationD30.txt
